##### Copyright 2025 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# T5Gemma 2 Inference Example

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Research/[T5Gemma_2]Example.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

We present T5Gemma 2, a family of encoder-decoder large langauge models adapted from Gemma 3, with strong multilingual, multimodal and long-context capabilities.

T5Gemma 2 was pretrained with UL2. The release includes 3 pretrained checkpoints: 270m-270m, 1b-1b, and 4b-4b. It can be used for in-context learning, and also be specialized to different downstream tasks.

In this notebook, we walk you through how to do inference with T5Gemma 2 270m-270m via Huggingface.


# Huggingface

## Install Library

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

## Model Loading

In [ ]:
import kagglehub

# Loading checkpoint from Kaggle (faster)
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoProcessor

model_id = "t5gemma-2-270m-270m"


# loading from kaggle
T5GEMMA_PATH = kagglehub.model_download(f"google/t5gemma-2/transformers/{model_id}")
model = AutoModelForSeq2SeqLM.from_pretrained(T5GEMMA_PATH)
processor = AutoProcessor.from_pretrained(T5GEMMA_PATH)

# loading from huggingface
# model = AutoModelForSeq2SeqLM.from_pretrained(f"google/{model_id}")
# processor = AutoProcessor.from_pretrained(f"google/{model_id}")


model = model.eval()


Loading weights:   0%|          | 0/911 [00:00<?, ?it/s]

## Text Sampling -- few-shot prompting

In [ ]:
prompts = """Translate the text from English to Chinese (zh_CN).

English: Hook the vacuum up to it, it helps keep the dust down, and you can prep your concrete that way also. We prefer to do it with the hand grinders ourself, so either way will work pretty good. Usually if it's got an epoxy coating already, the hand grinders work a little better, but this thing works really good too. So after we do that, we fix all the cracks, fix all the divots with our patch repair material, and then we grind them smooth. And then we clean the concrete and get it ready for the first coating. This is going to be a 100% solids epoxy, so it goes on in its different procedures, different stages.
Chinese (zh_CN): 把吸尘器连在上面，这样可以减少灰尘，你还可以顺便处理一下混凝土。我们更喜欢用手持式研磨机，两种方法的效果都不错。通常，如果表面已经有环氧树脂涂层的话，手持式研磨机的效果会更好一些，但这台机器的效果也很好。处理完之后，我们用修补材料把所有的裂缝和坑洼都补好，再把它们磨平。然后清洁混凝土表面，为涂刷第一层涂料做好准备。我们这里用的是 100% 固体环氧树脂，所以涂刷的时候会有不同的步骤和阶段。

English: The Zoroastrian text, Vendidad, states that Yima built an underground city on the orders of the god Ahura Mazda, to protect his people from a catastrophic winter. Much like the account of Noah in the Bible, Yima was instructed to collect pairs of the best animals and people as well as the best seeds in order to reseed the Earth after the winter cataclysm. This was before the last Ice Age, 110,000 years ago.
Chinese (zh_CN): 琐罗亚斯德教经典《万迪达德》中说，Yima 奉阿胡拉·马兹达神之命建造了一座地下城市，以保护他的人民躲避一场灾难性的寒冬。就像《圣经》中诺亚的故事一样，Yima 被指示收集最好的动物、人类以及最好的种子，以便在冬季灾难过后重新在地球上播种。这发生在最后一个冰河时代之前，也就是 11 万年前。

English: Okay, so let me explain. All right, so the problem is, if you look inside there... You see the wood siding? There's the old siding, and it butts up to the shingles there. And then I put this over it. And what happens is the dirt collects there, to that flashing.
Chinese (zh_CN): 好的，我来解释一下。问题是，你们看里面……看到木头墙板了吗？那是原来的墙板，紧挨着那边的瓦。然后我把这个盖在上面。结果灰尘就堆积在那儿，堆积到泛水板上。

English: Hey guys, Thunder E here, and welcome to the video you've been waiting for. I am talking about gaming on the ASUS ROG Phone 5. Now, the ROG Phone series is well known for its gaming powers, but in this video, we're going to find out if the ROG Phone 5 is truly taking back the crown as the king of gaming phones.
Chinese (zh_CN): 大家好，我是雷霆 E，欢迎收看大家期待已久的视频。今天要评测的是华硕 ROG Phone 5 的游戏性能。ROG Phone 系列手机一直以其强大的游戏性能而闻名，那么，ROG Phone 5 能否真正加冕“游戏手机之王”？我们拭目以待。

English: It is December 1997, and the Imperial Sugar Company is acquiring a new production site at Port Wentworth from Savannah Foods and Industries Incorporated. There is nothing really of note here. It was doing what businesses do, and that is acquiring to expand. The site has been home to food production and processing since the early 1900s. Savannah Industries Incorporated began construction of granulated sugar production facilities at Port Wentworth during the 1910s, completing it in 1917.
Chinese (zh_CN): 那是 1997 年 12 月，帝国糖业公司正从萨凡纳食品和工业有限公司手中收购位于温特沃斯港的一个新生产基地。这的确没什么值得注意的，它做的只是一家公司都会做的事情，那就是通过收购来扩张。该基地自 20 世纪初以来一直是食品生产和加工的场所。萨凡纳工业有限公司在 20 世纪 10 年代开始在温特沃斯港建造砂糖生产设施，并于 1917 年竣工。

English: Time for the Scotty Kilmer channel. Does your car have faded paint on it? Then stay tuned, because today I'm going to show you how to polish off faded paint. And all it takes is a bucket of water, a polisher, and a bottle of this Meguiar's Ultimate Compound.
Chinese (zh_CN):"""

model_inputs = processor(text=prompts, return_tensors="pt")

# 1=><eos>, 108=>\n\n
generation = model.generate(**model_inputs, max_new_tokens=100, do_sample=False, eos_token_id=[1, 108])

decoded = processor.batch_decode(generation)
decoded[0]

'<bos> 欢迎来到 Scotty Kilmer 的频道。你的车上有一点点褪色的漆？那就继续关注，今天我将告诉你如何擦掉褪色的漆。只需要一个桶的水，一个磨砂剂和一个瓶子，就可以完成这一步。\n\n'

## Text Sampling -- UL2 Infilling

*UL2 in T5Gemma2*

We use unused tokens from the tokneizer as the placeholder for masks in UL2.
The token starts from `<unused6237>` in an decreasing order, i.e, `<unused6236>, <unused6235> ...` We also added some noises during the training, so the infilling is not always robust. See example below.

In [ ]:
prompts = """A large language model (LLM) is a language model trained with self-supervised machine learning on a vast amount of text,
<unused6237>. The largest and most capable LLMs are generative
pre-trained transformers (GPTs) and provide the core capabilities of modern chatbots. LLMs can be fine-tuned for specific tasks
or guided by prompt engineering. These models <unused6236>"""

model_inputs = processor(text=prompts, return_tensors="pt")

generation = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)

decoded = processor.batch_decode(generation)
decoded[0]


'<bos><unused6237>speech, and images. It is a powerful tool for creating and understanding human language.\n<unused6236>\nare used in various fields, including text generation, natural language processing, and\nmachine learning.\nThe main goal of this project is to develop a large language model for text generation.<eos>'

## Image Understanding

In [ ]:
import requests
from PIL import Image
from transformers import AutoProcessor

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/bee.jpg"
image = Image.open(requests.get(url, stream=True).raw)

prompt = "<start_of_image> in this image, there is"
model_inputs = processor(text=prompt, images=image, return_tensors="pt")

generation = model.generate(**model_inputs, max_new_tokens=20, do_sample=False)

decoded = processor.batch_decode(generation)
decoded[0]

'<bos> a bumble bee in a flower bed.<eos>'

# END